In [1]:
'''
제 2유형 - 농작물 상태 분류하기

분류모델을 사용하여 macro F1 score값을 산출하시오.
예측값을 pred 변수에 담아 csv 파일로 제출하시오.

문제 : 농작물 분류

회귀 or 분류 : 분류 문제(범주 3개)
성능지표 : macro f1 score

기출 데이터셋 주요 정보
- 독립변수(x) : 비료 종류, 재배월 등
- 종속변수(y) : 농작물 상태(0:정상, 1:불량, 2:폐기)
- 데이터셋 : 불균형데이터(정상인 class가 대부분이고 불량,폐기는 소수 class)
- 결측치 : 없음

'''

'\n제 2유형 - 농작물 상태 분류하기\n\n분류모델을 사용하여 macro F1 score값을 산출하시오.\n예측값을 pred 변수에 담아 csv 파일로 제출하시오.\n\n문제 : 농작물 분류\n\n회귀 or 분류 : 분류 문제(범주 3개)\n성능지표 : macro f1 score\n\n기출 데이터셋 주요 정보\n- 독립변수(x) : 비료 종류, 재배월 등\n- 종속변수(y) : 농작물 상태(0:정상, 1:불량, 2:폐기)\n- 데이터셋 : 불균형데이터(정상인 class가 대부분이고 불량,폐기는 소수 class)\n- 결측치 : 없음\n\n'

In [2]:
############################# 복사 영역 ################################
# 데이터 생성(수정금지) 
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd

# 재현성을 위한 seed 설정
np.random.seed(42)

n_samples = 1000

# 독립변수 생성
month = np.random.randint(1, 13, size=n_samples)  # 재배한 월: 1~12 사이의 정수
regions = ["Region_1", "Region_2", "Region_3", "Region_4"]
region = np.random.choice(regions, size=n_samples)  # 4개의 지역 중 선택
fertilizers = ["A", "B", "C"]
fertilizer = np.random.choice(fertilizers, size=n_samples)  # 비료 종류 선택
soil_moisture = np.random.uniform(10, 100, size=n_samples)  # 토양 수분 (10~100)
temperature = np.random.uniform(15, 35, size=n_samples)  # 온도 (15~35℃)
rainfall = np.random.uniform(0, 200, size=n_samples)  # 강수량 (0~200 mm)
humidity = np.random.uniform(20, 100, size=n_samples)  # 습도 (20~100 %)
ph_level = np.random.uniform(4.0, 9.0, size=n_samples)  # 토양 pH (4.0~9.0)

# 종속변수 생성: 농작물 상태 (0: 정상, 1: 불량, 2: 폐기)
condition = np.random.choice([0, 1, 2], size=n_samples, p=[0.7, 0.2, 0.1])

# 데이터프레임 생성
data = pd.DataFrame({
    "Month": month,
    "Region": region,
    "Fertilizer": fertilizer,
    "Soil_Moisture": soil_moisture,
    "Temperature": temperature,
    "Rainfall": rainfall,
    "Humidity": humidity,
    "pH_Level": ph_level,
    "Condition": condition
})

# 데이터를 섞은 후 700개는 train, 300개는 test로 분할
data_shuffled = data.sample(frac=1, random_state=42)
train = data_shuffled.iloc[:700]
test = data_shuffled.iloc[700:]
y_test = test['Condition'] # 실제 시험에서는 주어지지 않습니다. 
test = test.drop(['Condition'], axis=1)

############################# 복사 영역 ################################


In [3]:
train.head(10)

,Month,Region,Fertilizer,Soil_Moisture,Temperature,Rainfall,Humidity,pH_Level,Condition
521,10,Region_4,C,53.740044,20.281240,104.680274,75.597611,6.349022,1
737,4,Region_1,B,68.303570,24.851381,129.120515,55.130453,6.844607,1
740,10,Region_2,B,55.277556,22.228076,156.378183,86.111225,7.697816,0
660,3,Region_1,C,61.617606,34.944758,57.636905,38.628685,6.372266,0
411,8,Region_2,A,84.635812,17.344078,101.186331,26.657845,4.090405,2
678,10,Region_1,C,72.327919,19.587316,172.332977,86.745984,5.128323,0
626,4,Region_3,C,52.799143,31.747342,189.854621,87.589127,8.670165,0
513,6,Region_4,B,20.850257,16.539433,121.017904,52.451621,8.842096,1
859,3,Region_2,C,97.841244,28.569999,14.672813,48.363857,7.726989,1
136,11,Region_1,C,88.545723,29.073996,10.114621,20.075473,6.687906,1


In [4]:
test.head(10)

,Month,Region,Fertilizer,Soil_Moisture,Temperature,Rainfall,Humidity,pH_Level
788,5,Region_4,A,66.944069,30.005315,23.990588,80.531724,6.060089
640,11,Region_4,C,94.963788,20.540250,110.694355,55.706109,7.112877
889,8,Region_2,A,52.642885,34.361528,13.520950,48.325165,5.118013
550,12,Region_1,B,48.747640,32.762364,129.114436,89.438370,7.750804
886,4,Region_3,A,32.194327,18.255820,148.458446,33.347886,8.807719
488,6,Region_3,C,33.184650,27.409476,56.066951,40.703393,4.187831
147,11,Region_3,C,75.655741,32.252809,174.137691,81.579730,8.113287
146,7,Region_4,B,49.316563,29.369079,67.490856,78.500047,7.329736
720,6,Region_4,B,14.520341,16.139575,175.950480,41.084118,4.919064
931,6,Region_2,C,51.369838,29.660002,163.773854,77.551052,4.823654


In [5]:
print(train.shape)
print(test.shape)

(700, 9)
(300, 8)


In [6]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 521 to 150
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Month          700 non-null    int64  
 1   Region         700 non-null    object 
 2   Fertilizer     700 non-null    object 
 3   Soil_Moisture  700 non-null    float64
 4   Temperature    700 non-null    float64
 5   Rainfall       700 non-null    float64
 6   Humidity       700 non-null    float64
 7   pH_Level       700 non-null    float64
 8   Condition      700 non-null    int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 54.7+ KB
None


In [7]:
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 788 to 102
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Month          300 non-null    int64  
 1   Region         300 non-null    object 
 2   Fertilizer     300 non-null    object 
 3   Soil_Moisture  300 non-null    float64
 4   Temperature    300 non-null    float64
 5   Rainfall       300 non-null    float64
 6   Humidity       300 non-null    float64
 7   pH_Level       300 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 21.1+ KB
None


In [8]:
print(train['Region'].unique())
print(test['Region'].unique())

['Region_4' 'Region_1' 'Region_2' 'Region_3']
['Region_4' 'Region_2' 'Region_1' 'Region_3']


In [9]:
print(train['Fertilizer'].unique())
print(test['Fertilizer'].unique())

['C' 'B' 'A']
['A' 'C' 'B']


In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['Region'] = le.fit_transform(train['Region'])
test['Region'] = le.transform(test['Region'])

train['Fertilizer'] = le.fit_transform(train['Fertilizer'])
test['Fertilizer'] = le.transform(test['Fertilizer'])


/var/folders/cf/wl7w208n3s3czgwpf7m_cqxr0000gn/T/ipykernel_61687/4157978316.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Region'] = le.fit_transform(train['Region'])
/var/folders/cf/wl7w208n3s3czgwpf7m_cqxr0000gn/T/ipykernel_61687/4157978316.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Fertilizer'] = le.fit_transform(train['Fertilizer'])


In [11]:
train.head()

,Month,Region,Fertilizer,Soil_Moisture,Temperature,Rainfall,Humidity,pH_Level,Condition
521,10,3,2,53.740044,20.281240,104.680274,75.597611,6.349022,1
737,4,0,1,68.303570,24.851381,129.120515,55.130453,6.844607,1
740,10,1,1,55.277556,22.228076,156.378183,86.111225,7.697816,0
660,3,0,2,61.617606,34.944758,57.636905,38.628685,6.372266,0
411,8,1,0,84.635812,17.344078,101.186331,26.657845,4.090405,2


In [12]:
test.head()

,Month,Region,Fertilizer,Soil_Moisture,Temperature,Rainfall,Humidity,pH_Level
788,5,3,0,66.944069,30.005315,23.990588,80.531724,6.060089
640,11,3,2,94.963788,20.540250,110.694355,55.706109,7.112877
889,8,1,0,52.642885,34.361528,13.520950,48.325165,5.118013
550,12,0,1,48.747640,32.762364,129.114436,89.438370,7.750804
886,4,2,0,32.194327,18.255820,148.458446,33.347886,8.807719


In [13]:
x = train.drop(columns='Condition')
y = train['Condition'] 

In [14]:
x

,Month,Region,Fertilizer,Soil_Moisture,Temperature,Rainfall,Humidity,pH_Level
521,10,3,2,53.740044,20.281240,104.680274,75.597611,6.349022
737,4,0,1,68.303570,24.851381,129.120515,55.130453,6.844607
740,10,1,1,55.277556,22.228076,156.378183,86.111225,7.697816
660,3,0,2,61.617606,34.944758,57.636905,38.628685,6.372266
411,8,1,0,84.635812,17.344078,101.186331,26.657845,4.090405
...,...,...,...,...,...,...,...,...
262,10,3,0,81.393522,20.174233,59.186406,45.242654,4.633186
610,3,1,0,60.193794,28.550632,166.522355,37.083428,6.151716
297,9,2,2,87.235935,16.369174,111.315686,94.605337,8.009129
414,7,0,1,56.570172,28.841473,199.549878,25.500313,7.097913


In [15]:
y

521    1
737    1
740    0
660    0
411    2
      ..
262    0
610    0
297    0
414    0
150    0
Name: Condition, Length: 700, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y,
                                                  stratify = y,
                                                  test_size=0.2,
                                                  random_state=2025)

In [17]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(560, 8) (560,)
(140, 8) (140,)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=2025)
rfc.fit(x_train,y_train)

RandomForestClassifier(random_state=2025)

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

pred = rfc.predict(x_val)

matrix = confusion_matrix(y_val, pred)
acc = accuracy_score(y_val, pred)
f1 = f1_score(y_val, pred, average='macro')

print(matrix)
print(acc)
print(f1)

[[97  3  0]
 [25  0  0]
 [14  1  0]]
0.6928571428571428
0.2740112994350283


In [20]:
from lightgbm import LGBMClassifier

In [21]:
lgbm_c = LGBMClassifier()
lgbm_c.fit(x_train,y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000127 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 955
[LightGBM] [Info] Number of data points in the train set: 560, number of used features: 8
[LightGBM] [Info] Start training from score -0.336472
[LightGBM] [Info] Start training from score -1.712816
[LightGBM] [Info] Start training from score -2.250399
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt

LGBMClassifier()

In [22]:
pred2 = lgbm_c.predict(x_val)

matrix2 = confusion_matrix(y_val,pred2)
acc2 = accuracy_score(y_val,pred2)
f1_2= f1_score(y_val,pred2, average='macro')

print(matrix2)
print(acc2)
print(f1_2)

[[88  6  6]
 [23  1  1]
 [14  1  0]]
0.6357142857142857
0.280942760942761


In [23]:
pred3 = rfc.predict(test)

matrix3 = confusion_matrix(y_test,pred3)
acc3 = accuracy_score(y_test,pred3)
f1_3= f1_score(y_test,pred3, average='macro')

print(matrix3)
print(acc3)
print(f1_3)

[[231   3   1]
 [ 39   0   0]
 [ 26   0   0]]
0.77
0.2900188323917137


In [24]:
pred3

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
pred = pd.DataFrame({'pred':pred3})
# pred.to_csv(index=False)
# df = pd.read_csv('./pred.csv')
# print(df.head(10))